# Se necessário utilize os pip's abaixo

In [ ]:
#!pip install streamlit
#!pip install pandas
#!pip install seaborn
#!pip install matplotlib.pyplot
#!pip install datetime
#!pip install pathlib
#!pip install plotly.express

In [ ]:
import pandas as pd
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from pathlib import Path
import warnings
import plotly.express as px
warnings.filterwarnings ('ignore') 

In [ ]:
DATA_PATH = Path('C:/Users/joaos/OneDrive/Área de Trabalho/DataSprints/Data/original data')

In [ ]:
df_2009 = pd.read_json(DATA_PATH / 'data-sample_data-nyctaxi-trips-2009-json_corrigido.json', lines=True)
df_2010 = pd.read_json(DATA_PATH / 'data-sample_data-nyctaxi-trips-2010-json_corrigido.json', lines=True)
df_2011 = pd.read_json(DATA_PATH / 'data-sample_data-nyctaxi-trips-2011-json_corrigido.json', lines=True)
df_2012 = pd.read_json(DATA_PATH / 'data-sample_data-nyctaxi-trips-2012-json_corrigido.json', lines=True)

# EDA

In [ ]:
display(df_2009.shape)
display(df_2010.shape)
display(df_2011.shape)
display(df_2012.shape)

In [ ]:
def basic_info(dataf):
    display(dataf.describe(include='all').transpose())
    print(f'\n{dataf.info()}')
    print(f'\nContagem de nulos(%):\n{(dataf.isnull().sum().sort_values(ascending=False)/dataf.shape[0])*100}')

In [ ]:
basic_info(df_2009)
print('\nFinal 2009--------------------------XX-----------------------------')
basic_info(df_2010)
print('\nFinal 2010--------------------------XX-----------------------------')
basic_info(df_2011)
print('\nFinal 2011--------------------------XX-----------------------------')
basic_info(df_2012)
print('\nFinal 2012--------------------------XX-----------------------------')

### Observações importantes

-> Através de uma breve pesquisa, foi verificado que a feature "surchase" é uma sobretaxa de 50 cents cobrada por áreas de NY, na qual em horários de pico é cobrado $1;
Referência: https://gothamist.com/news/50-cent-taxi-surcharge-goes-into-effect-sunday

-> A partir das descrições acima, nota-se que apesar de serem datasets de anos distintos, todos possuem a mesma quantidade de valores nulos nas features de "rate_code" e "store_and_fwd_flag".

-> Há número de passageiros setados como 0, teríamos que possuir melhor descrição dos dados para saber se há algum erro ou se é o trajeto considerado até o embarque do passageiro, por falta dessa informação, irei considerar como "trajeto até o embarque do passageiro", por tanto, será mantido para análises futuras.

-> Existem muitos valores faltantes em geolocalização onde o mesmo assume "0.00000", há também uma quantidade considerável de valores 0 na coluna "trip_distance"

-> Os anos 2011 e 2012 possuem meses faltantes, pode ser simplesmente por não haver corridas naquele mês ou ocorreu algum erro nos dados

## Pipeline opcional

In [ ]:
#%%writefile data.py

def solving_columns(dataf):
    dataf["payment_type"] = dataf["payment_type"].str.lower()
    dataf['pickup_datetime'] = pd.to_datetime(dataf['pickup_datetime']) 
    dataf['dropoff_datetime'] = pd.to_datetime(dataf['dropoff_datetime'])
    dataf['min_diff'] = (dataf['dropoff_datetime'] - dataf['pickup_datetime'])/ pd.Timedelta(minutes=1)
    dataf['pickup_day']= dataf['pickup_datetime'].dt.strftime("%Y-%m-%d").astype("datetime64")
    dataf['pickup_hour']= dataf['pickup_datetime'].dt.strftime("%H:%M:%S")
    dataf['dropoff_day']= dataf['dropoff_datetime'].dt.strftime("%Y-%m-%d").astype("datetime64")
    dataf['dropoff_hour']= dataf['dropoff_datetime'].dt.strftime("%H:%M:%S")
    dataf['Dia_semana'] = dataf['pickup_day'].dt.dayofweek
    dataf.drop(columns=['rate_code','store_and_fwd_flag','pickup_datetime','dropoff_datetime'], axis=1, inplace=True)
    return dataf

In [ ]:
#%%writefile respostas.py

def QM1(dataf):
    print('1. Qual a distância média percorrida por viagens com no máximo 2 passageiros')
    plt.figure(figsize=(10,6))
    sns.barplot(x=dataf[dataf['passenger_count'] <= 2] \
                .groupby(by=['passenger_count']) \
                .agg({'trip_distance':'mean'}) \
                .reset_index()['passenger_count'],
                
                y=dataf[dataf['passenger_count'] <= 2] \
                .groupby(by=['passenger_count']) \
                .agg({'trip_distance':'mean'}) \
                .reset_index()['trip_distance']);
    
    plt.ylabel('')

def QM2(dataf):
    print('2. Quais os 3 maiores vendors em quantidade total de dinheiro arrecadado;')
    plt.figure(figsize=(8, 6))
    sns.barplot(x=dataf.groupby(by=['vendor_id']).agg({'total_amount':'sum'}) \
                .sort_values('total_amount', ascending = False) \
                .reset_index()[:3]['vendor_id'], 
                
                y=dataf.groupby(by=['vendor_id']) \
                .agg({'total_amount':'sum'}) \
                .sort_values('total_amount', ascending = False) \
                .reset_index()[:3]['total_amount']/100,
                
                hue=dataf.groupby(by=['vendor_id']) \
                .agg({'total_amount':'sum'}) \
                .sort_values('total_amount', ascending = False) \
                .reset_index()[:3]['vendor_id'], palette="deep", dodge=False)
    
    plt.title('Valor total agrupado por vendor') 
    plt.ylabel('')
    plt.xlabel('')
    
def QM3(dataf):  
    print('3. Faça um histograma da distribuição mensal, nos 4 anos, de corridas pagas em dinheiro;')
    plt.figure(figsize=(15,8))
    sns.barplot(data = dataf[dataf['payment_type'] == 'cash'] \
                    .groupby(dataf['dropoff_day'].dt.to_period('M')) \
                    .agg({'total_amount':'sum'}) \
                    .reset_index(), x='dropoff_day', y='total_amount')
    plt.xticks(rotation=90)
    plt.title('Soma de pagamentos em dinheiro agrupado por mês')
    plt.legend(['Total'], fontsize=13)
    plt.ylabel('')
    plt.xlabel('');
    plt.show()
    
def QM4(dataf):
    print('4. Faça um gráfico de série temporal contando a quantidade de gorjetas de cada dia, nos últimos 3 meses de 2012.')
    fig = px.bar(x=dataf[(dataf['dropoff_day'].dt.year == 2012) & (dataf['dropoff_day'].dt.month >= 8)] \
                .groupby(by=dataf["dropoff_day"]) \
                .agg({'tip_amount':'sum'}) \
                .reset_index()['dropoff_day'],
                 
                 y=dataf[(dataf['dropoff_day'].dt.year == 2012) & (dataf['dropoff_day'].dt.month >= 8)] \
                .groupby(by=dataf["dropoff_day"]) \
                .agg({'tip_amount':'sum'}) \
                .reset_index()["tip_amount"], 
                 
                 color=dataf[(dataf['dropoff_day'].dt.year == 2012) & (dataf['dropoff_day'].dt.month >= 8)] \
                .groupby(by=dataf["dropoff_day"]) \
                .agg({'tip_amount':'sum'}) \
                .reset_index()["dropoff_day"].dt.month, 
                 
                title="Gorjeta por dia", 
                labels={'x':'day',
                        'y':'tip'}, 
                color_continuous_scale='portland', 
                opacity=0.7)
    fig.show()
    
def bonus1(dataf):
    print('Qual o tempo médio das corridas nos dias de sábado e domingo;')
    fig = plt.figure(figsize=(14,10))
    sns.lineplot(data=dataf[(dataf['Dia_semana'] == 5) | (dataf['Dia_semana'] == 6)], 
             x="pickup_day", y="min_diff", hue="Dia_semana",palette='colorblind', style="Dia_semana", legend=False)
    plt.axhline(dataf['min_diff'][(dataf['Dia_semana'] == 5) | (dataf['Dia_semana'] == 6)].mean(), color='r',
            linestyle='--')
    plt.title('Tempo de corridas no Sábado e Domingo') 
    plt.ylabel('quantidade em minutos')
    plt.xlabel('')
    plt.legend(['Sábado','Domingo', 'Média'])
    fig.show()

## Transformação dos dados em função

In [ ]:
all_dfs = [df_2009, df_2010, df_2011, df_2012]
dfFull_func = pd.concat(all_dfs).reset_index(drop=True)

In [ ]:
dfFull_func = solving_columns(dfFull_func)

In [ ]:
QM1(dfFull_func)

In [ ]:
QM2(dfFull_func)

In [ ]:
QM3(dfFull_func)

In [ ]:
QM4(dfFull_func)

In [ ]:
bonus1(dfFull_func)

## Transformação dos dados

In [ ]:
#após algumas análises, resolvi juntar os datasets

all_dfs = [df_2009, df_2010, df_2011, df_2012]
dfFull = pd.concat(all_dfs).reset_index(drop=True)

In [ ]:
#Drop das colunas com muitos valores nulos, não irá trazer valor para a análise
dfFull.drop(columns=['rate_code','store_and_fwd_flag'], axis=1, inplace=True)

In [ ]:
#Perceba que temos 6 tipos únicos de payment_type porém por conta do upper case

dfFull['payment_type'].value_counts()

In [ ]:
dfFull["payment_type"] = dfFull["payment_type"].str.lower()

In [ ]:
#tratando data
dfFull['pickup_datetime'] = pd.to_datetime(dfFull['pickup_datetime']) 
dfFull['dropoff_datetime'] = pd.to_datetime(dfFull['dropoff_datetime'])

In [ ]:
dfFull['min_diff'] = (dfFull['dropoff_datetime'] - dfFull['pickup_datetime'])/ pd.Timedelta(minutes=1)

In [ ]:
dfFull['pickup_day']= dfFull['pickup_datetime'].dt.strftime("%Y-%m-%d").astype("datetime64")
dfFull['pickup_hour']= dfFull['pickup_datetime'].dt.strftime("%H:%M:%S")
dfFull['dropoff_day']= dfFull['dropoff_datetime'].dt.strftime("%Y-%m-%d").astype("datetime64")
dfFull['dropoff_hour']= dfFull['dropoff_datetime'].dt.strftime("%H:%M:%S")

In [ ]:
dfFull.drop(columns=['pickup_datetime','dropoff_datetime'], axis=1, inplace=True)

In [ ]:
dfFull.duplicated().sum()

In [ ]:
dfFull.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(y=dfFull['passenger_count']) 
plt.title('Quantidade de passageiros por corrida') 
plt.show()

### Respostas

#### 1. Qual a distância média percorrida por viagens com no máximo 2 passageiros;

In [ ]:
df2passenger = dfFull[dfFull['passenger_count'] <= 2]

In [ ]:
#média geral com no máximo 2 passageiros
df2passenger['trip_distance'].mean()

In [ ]:
dfplot = df2passenger.groupby(by=['passenger_count']).agg({'trip_distance':'mean'}).reset_index()

In [ ]:
#média por quantidade de passageiros
plt.figure(figsize=(10,6))
sns.barplot(x=dfplot['passenger_count'], y=dfplot['trip_distance']);

#### 2. Quais os 3 maiores vendors em quantidade total de dinheiro arrecadado;

In [ ]:
top3vendor = dfFull.groupby(by=['vendor_id']).agg({'total_amount':'sum'}) \
    .sort_values('total_amount', ascending = False) \
    .reset_index()[:3]

In [ ]:
#reduzi a dimensão de total_amount para melhor leitura da diferença entre CMT e VTS
plt.figure(figsize=(8, 6))
sns.barplot(x=top3vendor['vendor_id'],y=top3vendor['total_amount']/100, hue=top3vendor['vendor_id'], palette="deep", dodge=False)
plt.title('Soma de total amount agrupado por vendor') 
plt.ylabel('')
plt.xlabel('');

#### 3. Faça um histograma da distribuição mensal, nos 4 anos, de corridas pagas em dinheiro;

In [ ]:
#Há corridas com inicio no mês X e final no mês Y, como o pagamento em dinheiro ocorre somente no final da corrida, irei utilizar a coluna de dropoff_day
dfFull[dfFull['payment_type'] == 'cash'] \
                .groupby(by=dfFull["dropoff_day"]) \
                .agg({'total_amount':'sum'}) \
                .reset_index()

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(data = dfFull[dfFull['payment_type'] == 'cash'] \
                .groupby(dfFull['dropoff_day'].dt.to_period('M')) \
                .agg({'total_amount':'sum'}) \
                .reset_index(), x='dropoff_day', y='total_amount')
plt.xticks(rotation=90)
plt.title('Soma de pagamentos em dinheiro agrupado por mês')
plt.legend(['Total'], fontsize=13)
plt.ylabel('')
plt.xlabel('');
plt.show()

#### 4. Faça um gráfico de série temporal contando a quantidade de gorjetas de cada dia, nos últimos 3 meses de 2012.

In [ ]:
dftip = dfFull[(dfFull['dropoff_day'].dt.year == 2012) & (dfFull['dropoff_day'].dt.month >= 8)] \
                .groupby(by=dfFull["dropoff_day"]) \
                .agg({'tip_amount':'sum'}) \
                .reset_index()

In [ ]:
fig = px.bar(dftip, x='dropoff_day', y="tip_amount", color=dftip["dropoff_day"].dt.month, title="Tip for day", \
            labels={'dropoff_day':'day'}, color_continuous_scale='portland', opacity=0.7)
fig.show()

#### Qual o tempo médio das corridas nos dias de sábado e domingo;

In [ ]:
#irei considerar agora o inicio das corridas corridas, mesmo que ela comece, por exemplo, às 23:59:59 de Domingo
# Domingo = 6 // Sábado = 5

dfFull['Dia_semana'] = dfFull['pickup_day'].dt.dayofweek

In [ ]:
dfFull[(dfFull['Dia_semana'] == 5) | (dfFull['Dia_semana'] == 6)].head()

In [ ]:
#tempo médio (em minutos) das corridas nos dias de sábado e domingo
dfFull['min_diff'][(dfFull['Dia_semana'] == 5) | (dfFull['Dia_semana'] == 6)].mean()

In [ ]:
fig = plt.figure(figsize=(14,10))
sns.lineplot(data=dfFull[(dfFull['Dia_semana'] == 5) | (dfFull['Dia_semana'] == 6)], 
             x="pickup_day", y="min_diff", hue="Dia_semana",palette='colorblind', style="Dia_semana", legend=False)
plt.axhline(dfFull['min_diff'][(dfFull['Dia_semana'] == 5) | (dfFull['Dia_semana'] == 6)].mean(), color='r',
            linestyle='--')
plt.title('Tempo de corridas no Sábado e Domingo') 
plt.ylabel('quantidade em minutos')
plt.xlabel('')
plt.legend(['Sábado','Domingo', 'Média'])
fig.show()